In [52]:
from enum import Enum
from pydantic import BaseModel, Field
from fastapi import FastAPI, status
from fastapi.encoders import jsonable_encoder
from fastapi.responses import JSONResponse
import uvicorn

## Enums

In [53]:
class ExperimentsType(str, Enum):
    TABULAR = 'tabular'
    IMAGE = 'image'

class DatasetType(str, Enum):
    CUSTOM = 'custom'
    SAMPLE = 'sample'

class DatasetName(str, Enum):
    CUSTOM = 'custom'
    IRIS = 'iris'
    SARS_COV = 'sars_cov'

class User(BaseModel):
    email: str

## Domain

In [54]:
import datetime
from kubernetes import client

class Dataset(BaseModel):
    name: DatasetName
    path: str
    volume: str
    target: str

In [55]:
class CustomDataset(Dataset):
    name: DatasetName = Field(const=DatasetName.CUSTOM, default=DatasetName.CUSTOM)
    pass

In [56]:
class SampleDataset(Dataset):
   name: DatasetName
   pass 


In [57]:
# class Experiments:
#     def __init__(self, name: str, type: ExperimentsType, namespace: str, dataset: Dataset, user: User):
#         self.id: str = type.value + '-' + datetime.datetime.now().strftime("%Y%m%d%H%M%S")
#         self.name: str = name
#         self.type: ExperimentsType = type
#         self.namespace: str = namespace
#         self.dataset: Dataset = dataset
#         self.user: User = user     
    
class Experiments(BaseModel):
    id: str = Field(const=datetime.datetime.now().strftime("%Y%m%d%H%M%S"), default=datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    name: str
    type: ExperimentsType
    namespace: str
    dataset: Dataset
    user: User

In [58]:
class ExperimentsTabular(Experiments):
    type: ExperimentsType = Field(const=ExperimentsType.TABULAR, default=ExperimentsType.TABULAR)
    pass

class ExperimentsImage(Experiments):
    type: ExperimentsType = Field(const=ExperimentsType.IMAGE, default=ExperimentsType.IMAGE)
    pass
    

## Experiments

In [118]:
d1 = CustomDataset(path='/dir1/dir2/csv', target='vartarget', volume='vol1')

In [120]:
d2 = SampleDataset(name=DatasetName.IRIS, path='/dir1/dir2/csv', target='vartarget', volume='vol2')

In [121]:
user = User(email='user1@example.com')

In [122]:
# Exp 1

exp1 = ExperimentsTabular(name='exp-tab-1', namespace='bra-dpmp', dataset=d1, user=user)
jsonable_encoder(exp1)

{'id': '20230331170846',
 'name': 'exp-tab-1',
 'type': 'tabular',
 'namespace': 'bra-dpmp',
 'dataset': {'name': 'custom',
  'path': '/dir1/dir2/csv',
  'volume': 'vol1',
  'target': 'vartarget'},
 'user': {'email': 'user1@example.com'}}

In [123]:
# Exp 2

exp2 = ExperimentsTabular(name='exp-tab-2', namespace='bra-dpmp', dataset=d2, user=user)
jsonable_encoder(exp2)

{'id': '20230331170846',
 'name': 'exp-tab-2',
 'type': 'tabular',
 'namespace': 'bra-dpmp',
 'dataset': {'name': 'iris',
  'path': '/dir1/dir2/csv',
  'volume': 'vol2',
  'target': 'vartarget'},
 'user': {'email': 'user1@example.com'}}

## Interfaces

In [129]:
!pip install zope.interface zope.component

     -------------------------------------- 69.3/69.3 kB 541.5 kB/s eta 0:00:00


In [59]:
from zope import interface, component

In [60]:
class IKatibService(interface.Interface):
    def create(exp: Experiments):
        pass

In [65]:
from zope import interface

@interface.implementer(IKatibService)
class KatibServiceImplementation:
    def create(exp: Experiments):
        return 'Charles' 
    
# Registry provider
component.provideUtility(KatibServiceImplementation())

In [66]:
ks = component.getUtility(IKatibService)

In [67]:
ks.create()

'Charles'

## Interface ABC

In [2]:
from abc import ABC, abstractmethod

In [47]:
class IUser(ABC):
    @abstractmethod
    def findAll(self) -> str:
        pass

In [44]:
class UserImpl:
    def findAll(self) -> str:
        return 'Charles'

In [45]:
IUser.register(UserImpl)

__main__.UserImpl

In [46]:
u = IUser
print(u.findAll())



None
